# Semantic Nearest Neighbors Graphs

Anton Antonov   
September 2024

-----

## Introduction

***Retrieval Augmented Generation (RAG)*** is demonstrated using semantic nearest neighbors graph.

1. Semantic graph for words 
    - Select a set of words
    - Find LLM embeddings for the those words
    - Find nearest neighbor graph for the obtained vectors

2. Nearest neighbor graphs
    - We use vector databases made in the "Creation" section in the following workflow diagram.
    - We create a semantic relationship graph
    - Merge vector databases
    - Make a the corresponding (bigger) graph

Using ["LLM::RetrievalAugmentedGeneration"](https://raku.land/zef:antononcube/LLM::RetrievalAugmentedGeneration).

-----

## Setup

In [7]:
use Data::Importers;
use LLM::Functions;
use XDG::BaseDirectory :terms;

use LLM::RetrievalAugmentedGeneration;
use LLM::RetrievalAugmentedGeneration::VectorDatabase;

use Data::Reshapers;
use Data::Summarizers;
use Math::Nearest;
use Math::DistanceFunctions::Native;
use Statistics::OutlierIdentifiers;

use NativeCall;

use Math::Nearest;
use Graph;
use JavaScript::D3;

### JavaScript

Here we prepare the notebook to visualize with JavaScript:

In [8]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

Verification:

In [9]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

Here we set a collection of visualization variables:

In [10]:
my $title-color = 'Ivory';
my $stroke-color = 'SlateGray';
my $tooltip-color = 'LightBlue';
my $tooltip-background-color = 'none';
my $background = '1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $vertex-size = 6;
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

{charge => {iterations => 4, strength => -30}, collision => {iterations => 4, radius => 50}, link => {distance => 30}}

-------

## Words semantic graph

```mermaid
flowchart LR
    WS[Words set selection] --> WE[Word embeddings]
    WE --> WNN
    WNN[Nearest neighbor graph<br>one neighbor] --> WCC[Connected component<br>to focus on]
    WCC --> WNN2[Nearest neighbor graph<br>two neighbors]
    WNN2 --> WGP[Graph plot]
    WE <-.-> LLM{{LLM}}
    
    subgraph WGraph[Words Graph]
        WS
        WE
        WNN
        WNN2
        WCC
        WGP
    end
```

-------

## Words graph making

Here is a set of words:

In [2]:
my @content = <angel apple ardvark bible car cat cherry chocolate cookie cow devil film horse house movie plum projector raccoon tiger tree>;

@content.elems

20

Here we specify an LLM-access configuration:

In [3]:
my $conf = llm-configuration('Gemini');
#my $conf = llm-configuration('LLaMA', model => 'llama-embedding');
$conf.Hash.elems

24

Embedding function:

In [4]:
my $format = 'values';
my $method = 'tiny';
#llama-embedding(@content, :$format, :$method);

tiny

Here we create semantic search index:

In [11]:
#my $vdbObjSmall = create-semantic-search-index(@content, e => $conf, name => 'words', embedder => {llama-embedding($_, :$format, :$method)} )
my $vdbObjSmall = create-semantic-search-index(@content, e => $conf, name => 'words')

VectorDatabase(:id("6877aa03-fb10-44a0-8a44-37121428cbbd"), :name("words"), :elements(20), :sources(20), :precision(num64))

Here we see the dimensions of the obtained vectors:

In [12]:
$vdbObjSmall.vectors».elems

{00.0 => 768, 01.0 => 768, 02.0 => 768, 03.0 => 768, 04.0 => 768, 05.0 => 768, 06.0 => 768, 07.0 => 768, 08.0 => 768, 09.0 => 768, 10.0 => 768, 11.0 => 768, 12.0 => 768, 13.0 => 768, 14.0 => 768, 15.0 => 768, 16.0 => 768, 17.0 => 768, 18.0 => 768, 19.0 => 768}

Here we find the embedding of a certain word (using the same LLM model as above):

In [13]:
my $vec = llm-embedding("coffee", e => $conf).head;
#my $vec = llama-embedding("coffee", :$format, :$method);
$vec.elems

768

Here we find the closest Nearest Neighbors (NNs) of that word:

In [14]:
my @nns = $vdbObjSmall.nearest($vec, 3, prop => 'label' ).map(*.Slip)

[07.0 05.0 08.0]

Here are the corresponding words:

In [15]:
$vdbObjSmall.items{@nns}

(chocolate cat cookie)

Here we find the corresponding NNs graph with 1 and 2 nns per vertex:

In [16]:
my ($gr1, $gr2) = [1, 2].map({ 
        # NNs graph edges
        my @edges = nearest-neighbor-graph(
            $vdbObjSmall.vectors.pairs, 
            $_, 
            method => 'Scan', 
            distance-function => &euclidean-distance, 
            format => 'dataset'
        );

        # Replace IDs with names
        @edges .= map({ $_<from> = $vdbObjSmall.items{$_<from>}; $_<to> = $vdbObjSmall.items{$_<to>}; $_ });
        
        # Make the graph
        Graph.new(@edges)
}).flat;

.say for ($gr1, $gr2)


Graph(vertexes => 20, edges => 14, directed => False)
Graph(vertexes => 20, edges => 27, directed => False)


Find 1-nns graph's connected components:

In [19]:
my @comps = $gr1.connected-components.sort(-*.elems)

[(projector bible movie film apple) (tree house cow horse car) (plum cookie chocolate cherry) (devil angel) (raccoon ardvark) (tiger cat)]

In [21]:
#%js

$gr2.edges(:dataset)
==> js-d3-graph-plot(
        :$background,
        highlight => [|@comps.head, |$gr1.subgraph(@comps.head).edges],
        width => 600,
        vertex-label-color => 'Ivory',
        edge-thickness => 2,
        vertex-size => 3,
        vertex-color => 'Blue',
        edge-color => 'SteelBlue',
        force => { charge => {strength => -200, iterations => 4}, collision => {iterations => 1, radius => 10} }
    )

------

## Semantic nearest neighbors graph

```mermaid
flowchart LR
    subgraph LocalVDB[Local Folder]
        direction LR
        A(Vector Database 1)
        B(Vector Database 2)
        C(Vector Database N)
    end

    subgraph Creation
        ID
        SD
        EV
        CD
    end

    subgraph NNGraph[Semantic Graph]
        D 
        E
        CC
        T 
        GP   
    end

    ID[Ingest document collection]
    SD[Split Documents]
    EV[Get LLM Embedding Vectors]
    CD[Create Vector Database]
    ID --> SD --> EV --> CD

    EV <-.-> LLMs
    
    CD -.- CArray[[CArray<br>representation]]

    CD -.-> |export| LocalVDB

    LocalVDB -.- JSON[[JSON<br>representation]]

    LocalVDB -.-> |import|D[Ingest Vector Database]
 
    D -.- CArray
    D --> E
    E[Nearest neighbor graph] --> CC[Connected Components]
    E -.- |nearest neighbors<br>distance function|CArray
    CC --> T[LLM-derived titles<br>per component]
    T --> GP[Graph plot]

    subgraph LLMs
        direction LR
        OpenAI{{OpenAI}}
        Gemini{{Gemini}}
        MistralAI{{MistralAI}}
        LLaMA{{LLaMA}}
    end
```

------

## Ingest vector databases

Skim vector databases from the default directory and show summaries:

In [22]:
#% html
my @field-names = <id name item-count dimension version llm-service llm-embedding-model created>;
vector-database-objects(f=>'hash', :flat)
==> { $_.map({ $_<created> = $_<file>.IO.created.DateTime.Str.subst('T',' ').substr(^19); $_}).sort(*<created>).reverse }()
==> to-html(:@field-names)

id name item-count dimension version llm-service llm-embedding-model created 6877aa03-fb10-44a0-8a44-37121428cbbd words 20 768 0 gemini embedding-001 2024-09-21 13:36:37 f00209fa-0f35-41bf-8597-744dae9aab5d No833 283 4096 0 llama text-embedding-3-small 2024-09-19 01:31:12 a6b12f80-0ed7-4125-9e2b-9f77c11a16f8 No833 441 4096 0 llama text-embedding-3-small 2024-09-19 01:26:58 32bd53cb-7171-4d26-9a4f-11de377d3af1 words 20 4096 0 llama text-embedding-3-small 2024-09-19 01:14:27 5097c865-7fad-43d9-b7ca-456220d754b7 words 20 768 0 gemini embedding-001 2024-09-18 14:08:44 045f467c-193f-4df6-bec3-790d6c83ca64 No833 441 768 0 gemini embedding-001 2024-09-16 22:21:25 d2effebc-2cef-4b2b-84ca-5dcfa3c1864b No747 283 1536 0 chatgpt text-embedding-3-small 2024-09-12 13:32:30 5cb40fbb-9f69-48ca-9fc1-03ec8059ed99 No747 283 768 0 gemini embedding-001 2024-09-12 13:32:23 44f19858-730e-4b96-86b7-81e701f9df8f No747 283 1536 0 chatgpt text-embedding-3-small 2024-09-12 13:32:18 266b20ca-d917-4ac0-9b0a-7c420625666c No833 441 1536 0 chatgpt text-embedding-3-small 2024-09-12 13:32:11

Get specific databases:

In [23]:
#my @vdbs = vector-database-objects(f=>'hash', :flat).grep({ $_<name> ∈ <No833 No747> && $_<llm-service> eq 'gemini' }).map({ create-vector-database(file => $_<file>) })

my @vdbs = vector-database-objects(f=>'hash', :flat).grep({ $_<id> ∈ <266b20ca-d917-4ac0-9b0a-7c420625666c d2effebc-2cef-4b2b-84ca-5dcfa3c1864b> }).map({ create-vector-database(file => $_<file>) })

[VectorDatabase(:id("d2effebc-2cef-4b2b-84ca-5dcfa3c1864b"), :name("No747"), :elements(283), :sources(284), :precision(num64)) VectorDatabase(:id("266b20ca-d917-4ac0-9b0a-7c420625666c"), :name("No833"), :elements(441), :sources(442), :precision(num64))]

Sample vectors:

In [24]:
.say for @vdbs.head.vectors.pick(3)

154.0 => NativeCall::Types::CArray[num64].new
227.0 => NativeCall::Types::CArray[num64].new
048.0 => NativeCall::Types::CArray[num64].new


Sample items:

In [25]:
#% html
@vdbs.head.items.pick(3).map({ <key value> Z=> $_.kv })».Hash.Array 
==> to-html(field-names => <key value>, align => 'left')

key value 088.0 Even the situation with David Grush is fantastic. He goes into a hearing. He says a bunch of completely batshit crazy stuff, right? Can we agree on that? All right. And then weeks later, some representatives go into a skiff and they say, well, it certainly seems like it confirms some of what Grush has been telling us. And you're thinking, okay, so you've separated the confirmation, which you did abstractly because it was inside of a skiff, but you can only talk to people who've emerged from the skiff who are willing to say vague things and the crazy claims. Now, what is this really all about? Nobody knows. Now, what I've been saying for about four years is there's 070.0 Well, you might have an argument with him and he'll solve the problem you've been working on for two years in an hour if it takes him that long. You have to understand how vertical human achievement can be. And this guy is at the very top of the human mind. 267.0 the Hamilton of, uh, U S historical, uh, fame. I'm very, I'm, I'm very bullish on pride in the Anglophone universe. And we've got to stop moping around and the uk is supposed to lead it's been a while there's a really awesome netflix series uh world war ii from the front lines i think it's called so they've used a combination of ai and archive footage to recolor and put into 4k this entire series and it's outstanding and there's one about the battle of britain and it's been a while i moved away from the uk and i had my problems with it and i tried for a long time to try and sort of nudge the culture as best i could from within my business or or whatever i was doing and then just thought i can't i'm

-----

## Combined databases graph

Here we join the selected databases:

In [26]:
my $vdbObj2 = vector-database-join(@vdbs)

VectorDatabase(:id("ada319ef-6afd-4a63-98ef-f4c478e1e2b3"), :name("No747-AND-No833"), :elements(724), :sources(726), :precision(num64))

Find a nearest neighbors graph:

In [27]:
my @edges2 = nearest-neighbor-graph($vdbObj2.vectors.pairs, 1, method => 'Scan', distance-function => &euclidean-distance, format => 'raku');

my $gr2 = Graph.new(@edges2)

Graph(vertexes => 724, edges => 565, directed => False)

In [28]:
my @comps2 = $gr2.connected-components.sort(-*.elems).head(8);

@comps2.elems

8

In [29]:
#% js
@edges2
==> js-d3-graph-plot(
        :$background,
        highlight => @comps2.map({[|$_, |$gr2.subgraph($_).edges]}),
        vertex-label-color => 'none',
        edge-thickness => 2,
        vertex-size => 4,
        vertex-color => 'Blue',
        width => 1100,
        height => 950,
        edge-color => 'Gray',
        vertex-color => 'DarkGray',
        #force => {charge => {strength => -3, iterations => 4}, collision => {radius => 14, iterations => 4}, link => {distance => 1};}
    )

-----

## Nearest neighbor graph

Take one of the vector databases:

In [30]:
my $vdbObj = @vdbs.head

VectorDatabase(:id("d2effebc-2cef-4b2b-84ca-5dcfa3c1864b"), :name("No747"), :elements(283), :sources(284), :precision(num64))

In [31]:
.say for $vdbObj.items.pick(3)

067.0 => to see other dogs? I don't understand. Your dog's been dead in the backyard for years, and you're still talking about how you know you're going to go take it hiking so there's been in 40 years basically no progress in string theory no meaningful no useful internal to string theory but functionally outside of that yeah i mean in that's 40 years of string theory. In 50 years, the standard model of particle theory hasn't moved. There are no young people who have ever walked on the moon, and there are no young theoretical physicists who have contributed to our picture of the universe in a way that's been confirmed. If it's the case that the underpinnings of string theory aren't accurate,
189.0 => whether it does descend into this sort of post for the next one that you want to do the post-apocalyptic blown out windows spring mattress in the back corner world where nothing is unifying given that what we spoke about for the you know first 90 minutes is the world is confusing it's har

Make the nearest neighbor graph for the vectors in the vector database:

In [32]:
my @edges = nearest-neighbor-graph($vdbObj.vectors.pairs, 1, method => 'Scan', distance-function => &euclidean-distance, format => 'raku')

[237.0 => 239.0 178.0 => 184.0 261.0 => 256.0 066.0 => 077.0 266.0 => 267.0 269.0 => 268.0 252.0 => 071.0 177.0 => 176.0 125.0 => 126.0 279.0 => 094.0 011.0 => 028.0 139.0 => 138.0 259.0 => 212.0 022.0 => 023.0 272.0 => 260.0 030.0 => 029.0 121.0 => 017.0 202.0 => 210.0 132.0 => 121.0 006.0 => 005.0 238.0 => 239.0 273.0 => 274.0 169.0 => 168.0 275.0 => 048.0 256.0 => 255.0 182.0 => 183.0 251.0 => 248.0 195.0 => 196.0 099.0 => 100.0 070.0 => 076.0 080.0 => 081.0 114.0 => 109.0 034.0 => 013.0 283.0 => 002.0 159.0 => 160.0 003.0 => 013.0 032.0 => 033.0 244.0 => 219.0 204.0 => 203.0 250.0 => 249.0 068.0 => 069.0 137.0 => 109.0 082.0 => 274.0 270.0 => 268.0 216.0 => 217.0 234.0 => 236.0 133.0 => 127.0 071.0 => 062.0 055.0 => 056.0 224.0 => 225.0 131.0 => 128.0 014.0 => 018.0 013.0 => 010.0 033.0 => 013.0 214.0 => 222.0 160.0 => 159.0 260.0 => 272.0 145.0 => 146.0 199.0 => 200.0 246.0 => 247.0 165.0 => 164.0 094.0 => 098.0 222.0 => 228.0 239.0 => 237.0 218.0 => 219.0 171.0 => 175.0 076.0 => 

Make the graph:

In [33]:
my $gr = Graph.new(@edges)

Graph(vertexes => 283, edges => 216, directed => False)

Get graph's connected components:

In [34]:
my @comps = $gr.connected-components.sort(-*.elems);
.say for @comps.head(12)

(119.0 087.0 097.0 110.0 111.0 049.0 161.0 163.0 278.0 277.0 189.0 279.0 094.0 098.0 083.0 082.0 274.0 273.0 047.0)
(132.0 121.0 017.0 009.0 026.0 025.0 012.0 039.0 032.0 033.0 034.0 040.0 027.0 004.0 010.0 013.0 003.0)
(166.0 167.0 144.0 143.0 141.0 142.0 170.0 038.0 140.0 037.0)
(091.0 095.0 090.0 089.0 093.0 096.0 092.0 102.0 085.0)
(155.0 149.0 195.0 196.0 169.0 168.0 148.0 146.0 145.0)
(058.0 066.0 077.0 150.0 070.0 076.0 072.0 054.0)
(227.0 226.0 242.0 223.0 225.0 243.0 224.0 122.0)
(084.0 067.0 080.0 081.0 275.0 053.0 048.0)
(075.0 252.0 073.0 078.0 079.0 071.0 062.0)
(244.0 231.0 220.0 219.0 218.0 217.0 216.0)
(264.0 266.0 267.0 265.0 271.0 258.0 257.0)
(160.0 158.0 159.0 157.0 156.0 015.0)


Example paragraph:

In [35]:
#% markdown

$vdbObj.items<170.0>

It was fun. It was silly. And it's just like the, the constant stream of moronic abuse. I don't even know how much of it is from humans. I think a lot of it's from bots. I think Elon is very misguided he has this idea of like anybody who shrinks from criticism or uh you know jokes uh it's too thin skin it's like you have no idea what your product is your product allows stalking you don't know how

LLM function for naming a set of paragraphs:

In [36]:
my &fNamer = llm-function({"Summarize the text into a very short sentence that has at most 8 words: \n\n $_"})

-> **@args, *%args { #`(Block|3845635738952) ... }

Example title finding:

In [37]:
@comps[5]

(058.0 066.0 077.0 150.0 070.0 076.0 072.0 054.0)

In [38]:
&fNamer( $vdbObj.items{|@comps[5]}.join("\n") )

String theory led by Edward Witten destroyed physics.

Find titles for some of the largest components:

In [39]:
my @titles = @comps.head(6).map({ &fNamer( $vdbObj.items{|$_}.join("\n") ) });

@titles.elems

6

In [40]:
.say for @titles

Society relies on complex questions without clear answers.
Harvard needs a leader with academic rigor and integrity.
Lewis Capaldi's rise to fame and the pressures of being well-known.
Secret organizations exist with UFO cover stories, hidden technology.
Respectful and reasonable, fame has its drawbacks.
String theory has destroyed physics and divided the scientific community.


Make rules for all components and titles:

In [41]:
my @res = do for ^@titles.elems -> $i {
    my @vals = @comps[$i].Array X~ ' : ' X~  @titles[$i];
    @comps[$i].Array Z=> @vals
}

my %rules = @res.map(*.Slip);

%rules.elems


72

Make graph highlight specification:

In [42]:
my @colors = <#1f77b4 #ff7f0e #2ca02c #d62728 #9467bd #8c564b #e377c2 #7f7f7f #bcbd22 #17becf>;

my $highlight = (@colors.head(6).Array Z=> @comps.head(6).map({ [ |$_.map({ %rules{$_} // $_ }), |$gr.subgraph($_).edges.map({ ( %rules{$_.key} // $vdbObj.items{$_.key} ) => ( %rules{$_.value} // $vdbObj.items{$_.value} ) }) ]})).Hash;

{#1f77b4 => [119.0 : Society relies on complex questions without clear answers. 087.0 : Society relies on complex questions without clear answers. 097.0 : Society relies on complex questions without clear answers. 110.0 : Society relies on complex questions without clear answers. 111.0 : Society relies on complex questions without clear answers. 049.0 : Society relies on complex questions without clear answers. 161.0 : Society relies on complex questions without clear answers. 163.0 : Society relies on complex questions without clear answers. 278.0 : Society relies on complex questions without clear answers. 277.0 : Society relies on complex questions without clear answers. 189.0 : Society relies on complex questions without clear answers. 279.0 : Society relies on complex questions without clear answers. 094.0 : Society relies on complex questions without clear answers. 098.0 : Society relies on complex questions without clear answers. 083.0 : Society relies on complex questions witho

Semantic graph:

In [43]:
#%js
my @edges2 = @edges.map({ ( %rules{$_.key} // $vdbObj.items{$_.key} ) => ( %rules{$_.value} // $vdbObj.items{$_.value} ) });

@edges2
==> js-d3-graph-plot(
        :$background,
        :$highlight,
        vertex-label-color => 'none',
        edge-thickness => 2,
        vertex-size => 4,
        vertex-color => 'Blue',
        width => 1100,
        height => 700,
        edge-color => 'Gray',
        vertex-color => 'DarkGray',
    )

Generate LLM based on one of components:

In [44]:
#% markdown

llm-synthesize([ 
    "Summarize the following text into a list of three-four points, each with no more than 8 words:",
    $vdbObj.items{|@comps[5].sort}.join(" "),
], e => $conf)

- String theory remains incomplete after 40 years.
- String theorists have made grandiose claims without delivering results.
- Edward Witten's dominance has stifled alternative theories.
- Witten's contributions both advanced and undermined physics.

-------

## References

### Articles

[AA1] Anton Antonov, 
["Outlier detection in a list of numbers"](https://rakuforprediction.wordpress.com/2022/05/29/outlier-detection-in-a-list-of-numbers/),
(2022),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

### Packages

[AAp1] Anton Antonov,
[WWW::OpenAI Raku package](https://github.com/antononcube/Raku-WWW-OpenAI),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[WWW::PaLM Raku package](https://github.com/antononcube/Raku-WWW-PaLM),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov,
[LLM::Functions Raku package](https://github.com/antononcube/Raku-LLM-Functions),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp4] Anton Antonov,
[LLM::Prompts Raku package](https://github.com/antononcube/Raku-LLM-Prompts),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp5] Anton Antonov,
[ML::FindTextualAnswer Raku package](https://github.com/antononcube/Raku-ML-FindTextualAnswer),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp6] Anton Antonov,
[Math::Nearest Raku package](https://github.com/antononcube/Raku-Math-Nearest),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp7] Anton Antonov,
[Math::DistanceFunctions Raku package](https://github.com/antononcube/Raku-Math-DistanceFunctions),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp8] Anton Antonov,
[Statistics::OutlierIdentifiers Raku package](https://github.com/antononcube/Raku-Statistics-OutlierIdentifiers),
(2022),
[GitHub/antononcube](https://github.com/antononcube).

## Videos

[CWv1] Chris Williamson,
["Eric Weinstein - Why Does The Modern World Make No Sense? (4K)"](https://www.youtube.com/watch?v=p_swB_KS8Hw),
(2024),
[YouTube/@ChrisWillx](https://www.youtube.com/@ChrisWillx).   
([transcript](https://podscripts.co/podcasts/modern-wisdom/747-eric-weinstein-why-does-the-modern-world-make-no-sense).)

[CWv2] Chris Williamson,
["Eric Weinstein - Are We On The Brink Of A Revolution? (4K)"](https://www.youtube.com/watch?v=PYRYXhU4kxM),
(2024),
[YouTube/@ChrisWillx](https://www.youtube.com/@ChrisWillx).   
([transcript](https://podscripts.co/podcasts/modern-wisdom/833-eric-weinstein-are-we-on-the-brink-of-a-revolution).)